# An Analysis of Political Contributions During the 2020 House of Representatives Election

In this part, you will obtain as much data as you can on the campaign contributions received by each candidate. This data is avaiable through the website https://www.opensecrets.org/.

### Part 1: Data Gathering

#### 1. Start by acquiring the data from Tennessee's 7th District, which is available at https://www.opensecrets.org/races/summary?cycle=2020&id=TN07&spec=N. If you click the "Download .csv file", you can get a csv for this district. However, we don't want to have to click this button across all districts. Instead, we'll use Python to help automate this process. Start by sending a get request to the download button URL, https://www.opensecrets.org/races/summary.csv?cycle=2020&id=TN07. Convert the result to a DataFrame.

In [1]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
import urllib3
# import re2
import regex
import re
import io
from requests.exceptions import HTTPError
from IPython.core.display import HTML


In [2]:
url = 'https://www.opensecrets.org/races/summary.csv?cycle=2020&id=TN07'

#Lets use try-except whenever we make any any http request.

#If we invoke .raise_for_status(), then Requests will raise an HTTPError for status codes between 400 and 600. 
#If the status code indicates a successful request, then the program will proceed without raising that exception.

try:
    response = requests.get(url)
    response.raise_for_status()
except HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    data = response.content.decode('utf8')
    df = pd.read_csv(io.StringIO(data))

#Lets populate the district ID column with TN07 so that we could use it later
df['DistIDCurr']='TN07'
df


,cid,FirstLastP,Rcpts,Spent,PACs,Indivs,Cand,Other,EndCash,LgIndivs,...,Result,CRPICO,State,IncCID,Incumbent,primarydate,DistIDCurr,capeye,sort,SmLgIndivsNote
0,N00041873,Mark Green (R),1194960.47,935486.67,171900.0,819151.42,0.0,203909.05,287888.55,819151.42,...,W,I,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,1,N
1,N00045536,Kiran Sreepada (D),206644.28,207190.98,4000.0,202644.28,0.0,0.00,0.00,179129.75,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,2,N
2,N00047077,Ronald Brown (I),1750.00,0.00,0.0,1750.00,0.0,0.00,9006.00,300.00,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,2,N
3,N00046592,Scott Vieira Jr (I),655.47,1048.51,10.0,45.00,35.0,565.47,-196.52,0.00,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,2,N
4,N00045535,Benjamin Estes (3),0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,...,,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN07,0,2,N


In [3]:
response

<Response [200]>

Raise for error if an error is encountered:

In [4]:
response.raise_for_status()

In [5]:
data

'cid,FirstLastP,Rcpts,Spent,PACs,Indivs,Cand,Other,EndCash,LgIndivs,SmIndivs,enddate,ReleaseDate,VotePercent,Result,CRPICO,State,IncCID,Incumbent,primarydate,DistIDCurr,capeye,sort,SmLgIndivsNote\nN00041873,Mark Green (R),1194960.47,935486.67,171900.0,819151.42,0.0,203909.05,287888.55,819151.42,0.0,2020-12-31 00:00:00 +0000,2021-04-01 00:00:00 +0000,69.93,W,I,Tennessee,"","",2020-08-06 00:00:00 +0000,TN07,0,1,N\nN00045536,Kiran Sreepada (D),206644.28,207190.98,4000.0,202644.28,0.0,0.0,0.0,179129.75,23514.53,2020-12-31 00:00:00 +0000,2021-04-01 00:00:00 +0000,27.33,L,C,Tennessee,"","",2020-08-06 00:00:00 +0000,    ,0,2,N\nN00047077,Ronald Brown (I),1750.0,0.0,0.0,1750.0,0.0,0.0,9006.0,300.0,1450.0,2020-12-31 00:00:00 +0000,2021-04-01 00:00:00 +0000,2.17,L,C,Tennessee,"","",2020-08-06 00:00:00 +0000,    ,0,2,N\nN00046592,Scott Vieira Jr (I),655.47,1048.51,10.0,45.0,35.0,565.47,-196.52,0.0,45.0,2020-10-14 00:00:00 +0000,2021-04-01 00:00:00 +0000,0.57,L,C,Tennessee,"","",2020-08-06 00:00:0

#### 2. Once you have working code for Tennessee's 7th District, expand on your code to capture all of Tennessee's districts into a single DataFrame. Make sure that you can distinguish which district each result came from. Export the results to a csv file.


In [6]:
#Lets define function to get the dataFrame for district

def getDistrictData(State_n_DistrictCode):
    url = f'https://www.opensecrets.org/races/summary.csv?cycle=2020&id={State_n_DistrictCode}'

    #Lets use try-except whenever we make any any http request.
    #If we invoke .raise_for_status(), then Requests will raise an HTTPError for status codes between 400 and 600. 
    #If the status code indicates a successful request, then the program will proceed without raising that exception.

    try:
        response = requests.get(url)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    else:
        data = response.content.decode('utf8')
        df = pd.read_csv(io.StringIO(data))

    #Lets populate the district ID column with TN07 so that we could use it later
    df['DistIDCurr'] = State_n_DistrictCode
    return df


#Lets define dictionary with state and district code.
districtList = ['01','02','03','04','05','06','07','08','09']
StateDistricts = {
    "state":'TN',
    "Districts": districtList
}

#lets define the Data Frame list which we can use later to concatenate all district data
frameList=[]

for i in range(0,len(StateDistricts['Districts'])):
    state_n_district_code = f'{StateDistricts["state"]}{StateDistricts["Districts"][i]}'
    frameList.append(getDistrictData(state_n_district_code))

#dist08 = getDistrictData('TN08')
#dist08

StateDistrictframe = pd.concat(frameList)
StateDistrictframe



,cid,FirstLastP,Rcpts,Spent,PACs,Indivs,Cand,Other,EndCash,LgIndivs,...,Result,CRPICO,State,IncCID,Incumbent,primarydate,DistIDCurr,capeye,sort,SmLgIndivsNote
0,N00046688,Diana Harshbarger (R),2126945.60,1869099.77,222800.00,359728.50,1461293.00,83124.10,257845.83,315489.10,...,W,O,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN01,0,2,N
1,N00046686,Blair Nicole Walsingham (D),140209.14,134994.55,1520.00,138689.14,0.00,0.00,5214.59,70085.20,...,L,O,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN01,0,2,N
2,N00047760,Steve Holder (I),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,,O,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN01,0,2,N
0,N00041594,Tim Burchett (R),1336275.75,878487.63,269535.00,1072845.61,0.00,-6104.86,593677.72,729831.26,...,W,I,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN02,0,1,N
1,N00041699,Renee Hoyos (D),812783.86,816793.15,3100.00,807459.01,0.00,2224.85,209.82,807459.01,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN02,0,2,N
2,N00047761,Matthew Campbell (I),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN02,0,2,N
0,N00030815,Chuck Fleischmann (R),1051653.39,381411.20,453858.46,603344.93,0.00,-5550.00,1880341.32,599059.93,...,W,I,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN03,0,1,N
1,N00046911,Meg Gorman (D),85843.21,77759.83,2671.60,81271.61,2000.00,-100.00,8083.38,50245.20,...,L,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN03,0,2,N
2,N00046589,Nancy Baxley (I),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN03,0,2,N
3,N00047762,Amber Hysell (I),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,,C,Tennessee,NaN,NaN,2020-08-06 00:00:00 +0000,TN03,0,2,N


In [7]:
type(df)

pandas.core.frame.DataFrame

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cid             5 non-null      object 
 1   FirstLastP      5 non-null      object 
 2   Rcpts           5 non-null      float64
 3   Spent           5 non-null      float64
 4   PACs            5 non-null      float64
 5   Indivs          5 non-null      float64
 6   Cand            5 non-null      float64
 7   Other           5 non-null      float64
 8   EndCash         5 non-null      float64
 9   LgIndivs        5 non-null      float64
 10  SmIndivs        5 non-null      float64
 11  enddate         4 non-null      object 
 12  ReleaseDate     5 non-null      object 
 13  VotePercent     5 non-null      float64
 14  Result          5 non-null      object 
 15  CRPICO          5 non-null      object 
 16  State           5 non-null      object 
 17  IncCID          0 non-null      float64

### 3. Once you have working code for all of Tennessee's districts, expand on it to capture all states and districts. 
    The number of districts for each state can be found at https://en.wikipedia.org/wiki/2020_United_States_House_of_Representatives_elections. You may also find the table of state abbreviations here helpful: https://en.wikipedia.org/wiki/List_of_U.S._state_and_territory_abbreviations. Export a csv file for each state.

Gathering Lists of US State Abbreviations and Number of Districts for Each State:

In [9]:
# https://en.wikipedia.org/wiki/2020_United_States_House_of_Representatives_elections
# Number of representatives for each state
# scrape wiki page - data strings from there and use those strings to interpolate the url through 

wiki_rep_url = 'https://en.wikipedia.org/wiki/2020_United_States_House_of_Representatives_elections'

r = requests.get(wiki_rep_url)
wiki_rep_soup = BeautifulSoup(r.text, features="html.parser")

table_html_rep_wiki = str(wiki_rep_soup.find('table', attrs={'class':['wikitable', 'sortable jquery-tablesorter'], 'style':'text-align:center'}))
HTML(table_html_rep_wiki)

In [10]:
wiki_rep_df = pd.read_html(io.StringIO(str(table_html_rep_wiki)))[0]
wiki_rep_df 

State Total seats Democratic        Republican       
             State Total seats      Seats Change      Seats Change
0          Alabama           7          1    NaN          6    NaN
1           Alaska           1          0    NaN          1    NaN
2          Arizona           9          5    NaN          4    NaN
3         Arkansas           4          0    NaN          4    NaN
4       California          53         42    4.0         11    4.0
5         Colorado           7          4    NaN          3    NaN
6      Connecticut           5          5    NaN          0    NaN
7         Delaware           1          1    NaN          0    NaN
8          Florida          27         11    2.0         16    2.0
9          Georgia          14          6    1.0          8    1.0
10          Hawaii           2          2    NaN          0    NaN
11           Idaho           2          0    NaN          2    NaN
12        Illinois          18         13    NaN          5    NaN
13         Indiana           9          2    NaN          7    NaN
14            Iowa           4          1    2.0          3    2.0
15          Kansas           4          1    NaN          3    NaN
16        Kentucky           6          1    NaN          5    NaN
17       Louisiana           6          1    NaN          5    NaN
18           Maine           2          2    NaN          0    NaN
19        Maryland           8          7    NaN          1    NaN
20   Massachusetts           9          9    NaN          0    NaN
21        Michigan          14          7    NaN          7    1.0
22       Minnesota           8          4    1.0          4    1.0
23     Mississippi           4          1    NaN          3    NaN
24        Missouri           8          2    NaN          6    NaN
25         Montana           1          0    NaN          1    NaN
26        Nebraska           3          0    NaN          3    NaN
27          Nevada           4          3    NaN          1    NaN
28   New Hampshire           2          2    NaN          0    NaN
29      New Jersey          12         10    1.0          2    1.0
30      New Mexico           3          2    1.0          1    1.0
31        New York          27         19    2.0          8    2.0
32  North Carolina          13          5    2.0          8    2.0
33    North Dakota           1          0    NaN          1    NaN
34            Ohio          16          4    NaN         12    NaN
35        Oklahoma           5          0    1.0          5    1.0
36          Oregon           5          4    NaN          1    NaN
37    Pennsylvania          18          9    NaN          9    NaN
38    Rhode Island           2          2    NaN          0    NaN
39  South Carolina           7          1    1.0          6    1.0
40    South Dakota           1          0    NaN          1    NaN
41       Tennessee           9          2    NaN          7    NaN
42           Texas          36         13    NaN         23    NaN
43            Utah           4          0    1.0          4    1.0
44         Vermont           1          1    NaN          0    NaN
45        Virginia          11          7    NaN          4    NaN
46      Washington          10          7    NaN          3    NaN
47   West Virginia           3          0    NaN          3    NaN
48       Wisconsin           8          3    NaN          5    NaN
49         Wyoming           1          0    NaN          1    NaN
50           Total         435        222   13.0        213   14.0

In [11]:
wiki_rep_df_2Columns = wiki_rep_df[['State', 'Total seats']]
wiki_rep_df_2Columns

,State,Total seats
,State,Total seats
0,Alabama,7
1,Alaska,1
2,Arizona,9
3,Arkansas,4
4,California,53
5,Colorado,7
6,Connecticut,5
7,Delaware,1
8,Florida,27


In [12]:
wiki_rep_df_2Columns_SingleLevel = wiki_rep_df_2Columns.to_csv(header=None,index=False)
wiki_rep_df_2Columns_SingleLevel_df = pd.read_csv(io.StringIO(wiki_rep_df_2Columns_SingleLevel), names=['US State', 'Number of Districts'])
wiki_rep_df_2Columns_SingleLevel_df

,US State,Number of Districts
0,Alabama,7
1,Alaska,1
2,Arizona,9
3,Arkansas,4
4,California,53
5,Colorado,7
6,Connecticut,5
7,Delaware,1
8,Florida,27
9,Georgia,14


In [13]:
StateAbbrev_url = 'https://www.worldatlas.com/geography/usa-states.html'

r = requests.get(StateAbbrev_url)
StateAbbrev_soup = BeautifulSoup(r.text, features="html.parser")

table_html_StateAbbrev = str(StateAbbrev_soup.find('table'))
HTML(table_html_StateAbbrev)

US State,Abbreviation
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA
Colorado,CO
Connecticut,CT
Delaware,DE
Florida,FL
Georgia,GA


In [14]:
StateAbbrev_df = pd.read_html(io.StringIO(str(table_html_StateAbbrev)))[0]
StateAbbrev_df

,US State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,Florida,FL
9,Georgia,GA


In [20]:
Merged_StateAbbrev_n_DistrictCode_df = pd.merge(StateAbbrev_df, wiki_rep_df_2Columns_SingleLevel_df, on = "US State", how = "inner").drop(['US State'], axis=1)
Merged_StateAbbrev_n_DistrictCode_df

,Abbreviation,Number of Districts
0,AL,7
1,AK,1
2,AZ,9
3,AR,4
4,CA,53
5,CO,7
6,CT,5
7,DE,1
8,FL,27
9,GA,14


In [21]:
Merged_StateAbbrev_n_DistrictCode_df['Number of Districts'] = Merged_StateAbbrev_n_DistrictCode_df['Number of Districts'].astype(str).str.zfill(2)
Merged_StateAbbrev_n_DistrictCode_df

,Abbreviation,Number of Districts
0,AL,07
1,AK,01
2,AZ,09
3,AR,04
4,CA,53
5,CO,07
6,CT,05
7,DE,01
8,FL,27
9,GA,14


pandas.core.series.Series

### 4. Finally, combine all of the data you've gathered together into a single DataFrame.